In [1]:
import numpy as np
import pandas as pd
import h5py

from osgeo import gdal
from osgeo import gdal_array
from osgeo import osr

from glob import glob
#from os import listdir
#from os.path import splitext
import logging
import os

In [2]:
dir_img = '//mnt/sda1/tiles_2018_wien/'

In [3]:
sufix = 'ortho/'
ids_ortho = [os.path.splitext(file)[0] for file in os.listdir(dir_img+sufix) if not file.startswith('.')]
sufix = 'ground_truth/'
ids_ground_truth = [os.path.splitext(file)[0] for file in os.listdir(dir_img+sufix) if not file.startswith('.')]

In [4]:
len(ids_ground_truth)

19305

In [5]:
ids_ortho = [dir_img + 'ortho/tile_ortho_' + str(i) + '.tif' for i in range(400,500)]
ids_ground_truth = [dir_img + 'ground_truth/tile_ground_truth_' + str(i) + '.tif' for i in range(400,500)]

In [6]:
ids_ortho[0]

'//mnt/sda1/tiles_2018_wien/ortho/tile_ortho_400.tif'

In [7]:
from osgeo import gdalconst

def tif2array(input_file, dtype=np.uint8):
    """
    read GeoTiff and convert to numpy.ndarray.
    Inputs:
        input_file (str) : the name of input GeoTiff file.
    return:
        image(np.array) : image for each bands
        dataset : for gdal's data drive.
    """
    dataset = gdal.Open(input_file, gdal.GA_ReadOnly)
    
    if dataset is None:
        return None
    
    # Allocate our array using the first band's datatype
    image_datatype = dataset.GetRasterBand(1).DataType
    image = np.zeros((dataset.RasterYSize, dataset.RasterXSize, dataset.RasterCount),
                     dtype=dtype)
    
    # Loop over all bands in dataset
    for b in range(dataset.RasterCount):
        # Remember, GDAL index is on 1, but Python is on 0 -- so we add 1 for our GDAL calls
        band = dataset.GetRasterBand(b + 1)
        # Read in the band's data into the third dimension of our array
        image[:, :, b] = band.ReadAsArray()#buf_type=gdalconst.GDT_Byte)
        
    #image = image[2:-2,2:-2,:]
        
    return image

In [8]:
def cut_img(img, x, y):
    # set pixel sizes
    x_i, y_i, z_i = img.shape
    # dict to store the sliceing information
    d = {}
    
    for var, var_i, key in [(x, x_i, 'x'), (y, y_i, 'y')]:
        # if image pixel size is grater than the target pixel size
        if (var_i > var):
            # if even cut same amount of pixels from both sides
            if var_i%2 == 0:
                sub = int(var_i/2 - var/2)
                d[key+'0'] = sub
                d[key+'1'] = sub
            # if odd cut 1 pixel more from right/bottom
            else:
                sub = int(var_i/2 - var/2)
                d[key+'0'] = sub
                d[key+'1'] = sub + 1
        else:
            print('image too small')
    # cut image
    img = img[d['x0']:-d['x1'],d['y0']:-d['y1']]
    
    return img

In [9]:
# pass ids_ortho, ids_dsm, ids_dtm, ids_slope
def create_array(ids, dtype):
    
    imgs = []
    
    if dtype == np.uint8:
        # add all
        for i in ids:
            # load image to numpy array
            img = tif2array(i, np.uint8)
            # cut into right shape
            img = cut_img(img, 512, 512)
            # append array to list
            imgs.append(img)
            
        # convert list with arrays to numpy array
        arr = np.stack(imgs, axis=0)
        print(arr.shape)
            
        # calculate mean
        #mean = np.mean(arr, dtype='float32')
        # calculate standard deviation
        #std = np.std(arr, dtype='float32')
    
    
    
    
    else:
        # add all
        for i in ids:
            
            # load image to numpy array
            img = tif2array(i, np.float32)
            # cut into right shape
            img = cut_img(img, 512, 512)
            # append array to list
            imgs.append(img)
            x, y, z = img.shape

        # convert list with arrays to numpy array
        arr = np.stack(imgs, axis=0)
        arr[arr < 0] = np.nan
        print(arr.shape)

        # calculate mean
        #mean = np.nanmean(arr, dtype='float32')
        # calculate standard deviation
        #std = np.nanstd(arr, dtype='float32')
        # convert nan to 0
        arr = np.nan_to_num(arr)
    
    # normalize data
    #arr = (arr - mean) / std
    
    return arr#, mean, std

In [10]:
arr_ortho = create_array(ids_ortho, np.uint8)
arr_ground_truth = create_array(ids_ground_truth, np.uint8)

(100, 512, 512, 4)
(100, 512, 512, 1)


In [58]:
for i in range(100):
    if tif2array(arr_ortho[i]).sum() != 0:
        img0 = arr_ortho[i, :256, :256]
        img1 = arr_ortho[i, 256:, :256]
        img2 = arr_ortho[i, :256, 256:]
        img3 = arr_ortho[i, 256:, 256:]
        
        arr_ground_truth
        

0 0
1 0
2 0
3 0
4 0
5 0
6 0
7 0
8 0
9 0
10 0
11 0
12 0
13 0
14 0
15 0
16 0
17 0
18 0
19 0
20 0
21 0
22 0
23 0
24 0
25 0
26 0
27 0
28 0
29 0
30 0
31 0
32 0
33 0
34 0
35 0
36 0
37 0
38 0
39 0
40 0
41 0
42 0
43 0
44 0
45 0
46 0
47 0
48 0
49 0
50 0
51 0
52 0
53 0
54 0
55 0
56 0
57 0
58 0
59 0
60 0
61 0
62 0
63 0
64 0
65 0
66 0
67 0
68 0
69 0
70 0
71 0
72 0
73 0
74 0
75 0
76 0
77 0
78 0
79 0
80 0
81 0
82 0
83 0
84 0
85 0
86 0
87 0
88 0
89 0
90 0
91 0
92 0
93 0
94 0
95 0
96 0
97 0
98 0
99 0
100 0
101 0
102 0
103 0
104 0
105 0
106 0
107 0
108 0
109 0
110 0
111 0
112 0
113 0
114 0
115 0
116 0
117 0
118 0
119 0
120 0
121 0
122 0
123 0
124 0
125 0
126 0
127 0
128 0
129 0
130 0
131 0
132 0
133 0
134 0
135 0
136 0
137 0
138 0
139 0
140 0
141 0
142 0
143 0
144 0
145 0
146 0
147 0
148 0
149 0
150 0
151 0
152 0
153 0
154 0
155 0
156 0
157 0
158 0
159 0
160 0
161 0
162 0
163 0
164 0
165 0
166 0
167 0
168 0
169 0
170 0
171 0
172 0
173 0
174 0
175 0
176 0
177 0
178 0
179 0
180 0
181 0
182 0
183 0
184 0


882 93027269
883 96751619
884 107662072
885 108497711
886 95508800
887 92616533
888 100032890
889 99310300
890 101125348
891 96292788
892 90377130
893 100268849
894 101827082
895 102788596
896 89127430
897 99341933
898 94192959
899 99559711
900 101243356
901 99859583
902 95346381
903 86791765
904 85200578
905 93718574
906 94798082
907 119408037
908 108539200
909 96700113
910 90210330
911 104684399
912 105930350
913 89757367
914 92219333
915 99931495
916 82414097
917 99897546
918 99576817
919 99583730
920 90424354
921 97753120
922 105375415
923 105488676
924 103640856
925 102316863
926 101223446
927 92157231
928 93389983
929 98747845
930 107632130
931 123927413
932 86888152
933 98503443
934 91472085
935 99348726
936 102101863
937 106722200
938 107100274
939 108774719
940 96909988
941 97792608
942 104145949
943 92833275
944 91019318
945 92875341
946 103579631
947 99779111
948 130015272
949 114931975
950 111924986
951 100896773
952 103636558
953 90909886
954 109526763
955 105838552
956 10

In [31]:
arr_ortho[:, 256:, :256].shape

(100, 256, 256, 4)

In [21]:
arr_ground_truth[50, 256:, :256].sum()

89867

In [12]:
# create mask
m = np.ma.make_mask(arr_ground_truth)

In [13]:
# apply mask
arr_ortho *= m

In [14]:
# create 256
arr_ortho_256 = np.concatenate([arr_ortho[:, :256, :256], \
arr_ortho[:, 256:, :256], \
arr_ortho[:, :256, 256:], \
arr_ortho[:, 256:, 256:]], axis=0)

arr_ground_truth_256 = np.concatenate([arr_ground_truth[:, :256, :256], \
arr_ground_truth[:, 256:, :256], \
arr_ground_truth[:, :256, 256:], \
arr_ground_truth[:, 256:, 256:]], axis=0)

In [15]:
arr_ortho_256.shape

(400, 256, 256, 4)

In [16]:
arr_ground_truth_256.shape

(400, 256, 256, 1)

In [17]:
arr_ortho_256[100].sum()

0

In [18]:
arr_ground_truth_256[100].sum()

178076

In [19]:
arr_ortho_256.shape[0]

400

In [20]:
# get indices with just zeros
idx_delete = []
for i in range(400):
    if arr_ortho_256[i].sum() == 0 or arr_ground_truth_256[i].sum() == 0:
        idx_delete.append(i)

In [21]:
len(idx_delete)

144

In [27]:
# delete images with just zeros
arr_ortho_256 = np.delete(arr_ortho_256, idx_delete, axis=0)
arr_ground_truth_256 = np.delete(arr_ground_truth_256, idx_delete, axis=0)

In [29]:
#ds_file_name  = "/home/philipp/Data/edin_dataset/dataset_512_1.h5"
ds_file_name  = "/mnt/sda1/dataset_256_0.h5"

tile_size = 256

#ortho_shape = (len(arr_ortho), 512, 512, 4)
#dsm_shape = (len(arr_dsm), 512, 512, 1)
ortho_shape = (arr_ortho_256.shape[0], tile_size, tile_size, 4)
gt_shape = (arr_ground_truth_256.shape[0], tile_size, tile_size, 1)

with h5py.File(ds_file_name, 'w') as hf:
    hf.create_dataset('x_ortho', data=arr_ortho_256, shape=ortho_shape, dtype=np.uint8, chunks=(100,tile_size,tile_size,4))
    hf.create_dataset('y_ground_truth', data=arr_ground_truth_256, shape=gt_shape, dtype=np.uint8, chunks=(100,tile_size,tile_size,1))

In [26]:
400-144

256